# Importing required libraries

In [ ]:

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


# Connecting to OpenAI

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("Key not found")
else:
    print(f'Key Found: {api_key}')

In [ ]:
openAI = OpenAI()

## Lets check preview of our frontier model is working

In [ ]:
msg = " Hey Gpt! How are you today ?"

response = openAI.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {
            "role" : "user",
            "content" : msg
        }
    ]
)

print(response.choices[0].message.content)

# Lets create a website class for scrapping

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):

        self.url = url
        response = requests.get(url, headers = headers)
        res = BeautifulSoup(response.content, 'html.parser')
        self.title = res.title.string if res.title else "No title found"

        for trash in res.body(["script", "style", "img", "input"]):
            trash.decompose()

        self.text = res.get_text(separator = "\n", strip = True)
                                    


In [ ]:
web = Website("https://www.deeplearning.ai/")
print(web.title)

In [ ]:
print(web.text)

# Giving prompts (System & User)

In [ ]:
system_prompt = " You are an assistant that analyzes the contents of a website \
and provides a short summary "

In [ ]:
def user(website):
    
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown.\n\n"
    user_prompt += website.text

    return user_prompt
    

# Automating the workflow

In [ ]:
def api_msg(website):

    return [
        {"role" : "system", "content" : system_prompt},
        {"role" : "user", 'content' : user(website)}
    ]

In [ ]:
def summarize(url):
    website = Website(url)
    response = openAI.chat.completions.create(
        model = "gpt-4o-mini",
        messages = api_msg(website)
    )

    return response.choices[0].message.content

In [ ]:
#lets call the workflow

summary = summarize("https://www.deeplearning.ai/")
summary

In [ ]:
# lets display output in markdown
def clean_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
clean_summary("https://www.deeplearning.ai/")

In [ ]:
clean_summary("https://www.apple.com/")